In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import smart_open
from multiprocessing import cpu_count
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="utf-8") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus("/content/drive/MyDrive/sharing/datasets/avito/preprocessed_data/prepr_descriptions.csv"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
import numpy as np
#print(train_corpus[:10])
#print(np.mean([len(test_corpus[i][0]) for  i in range(0, len(test_corpus))]))
#model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=3, epochs=40) #first version, +- что-то умеет, но раз на раз не приходится, тут надо как-то хорошо подобрать параметры и найти больше данных мб
#model = gensim.models.doc2vec.Doc2Vec(vector_size=10, min_count=2, epochs=30) #second verison, сейчас смотрю на нее, не станет ли получше(отвратительно)
#model.build_vocab(train_corpus)
model = gensim.models.doc2vec.Doc2Vec(dm = 1, workers = -1, window=10, min_count=1, vector_size=150, sample=1e-4, negative=2, epoch = 40)
model.build_vocab(train_corpus)


In [18]:
model.train(train_corpus, total_examples=model.corpus_count, epochs = max_epochs)

[('демисезоть', 0.35571804642677307),
 ('застегивать', 0.3327648639678955),
 ('варавва', 0.3326587677001953),
 ('simon', 0.3178480863571167),
 ('проткнуть', 0.31445983052253723),
 ('никуда', 0.3144182860851288),
 ('воротникать', 0.31277555227279663),
 ('точечнгый', 0.3086700439453125),
 ('феном', 0.3063196539878845),
 ('сабик', 0.3060757517814636)]

In [14]:
!git clone https://github.com/xianyi/OpenBLAS.git

Cloning into 'OpenBLAS'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (79/79), done.
^C


NameError: ignored

In [ ]:
model.save("/content/drive/MyDrive/sharing/models/doc2vec_2version.model")

In [41]:
test_corpus = list(read_corpus("/content/drive/MyDrive/sharing/datasets/avito/preprocessed_data/prepr_descriptions.csv", tokens_only =True))
#model = gensim.models.Doc2Vec.load("/content/drive/MyDrive/sharing/models/doc2vec.model")

In [42]:
import random
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
print("query = ", test_corpus[doc_id], doc_id)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

print("First 10:\n")
for index in range(10):
  print(sims[index], ' '.join(train_corpus[sims[index][0]].words)) # мб мало данных

query =  ['шуруповерт', 'bosch', 'ixo', 'upgrade', 'xxxxxxxxxx', 'продавать', 'совершенно', 'новый', 'не', 'распечатывать', 'жестяной', 'коробка'] 84912
Test Document (84912): «шуруповерт bosch ixo upgrade xxxxxxxxxx продавать совершенно новый не распечатывать жестяной коробка»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d150,n2,w10,s0.0001):

MOST (150720, 0.3918375074863434): «испания на майский вылет на ночь отель serhs oasis park от xxxxx рубль завтрак стоимость тур входить прямой перелет трансфер проживание двухместный номер быть предложение по другой отель звонить приезжать мы подберть для вы лучший вариант по самый выгодный цена возможный тур на другой дата количество день быть предложение по другой направление ждать вы наш офис станция метро купчино пр просвещение»

MEDIAN (6786, 0.00015948398504406214): «рабочий дль работа офисе интернуть видео самовывоз из росляково»

LEAST (308134, -0.38459834456443787): «угловой шкаф купе делать на заказ хороший состояние размер высота 

In [ ]:
#http://zabaykin.ru/?p=639

print(len(train_corpus))
?gensim.models.Doc2Vec
print(test_corpus[0], "----->", train_corpus[0][0])
print(test_corpus[:10])

489517
['картина'] -----> ['гобелен', 'размер']
[['картина'], ['стул', 'из', 'прессовать', 'кожа'], ['домашний', 'миня', 'баня'], ['эксклюзивный', 'коллекция', 'книга', 'трансаэро', 'плюс', 'подарок'], ['ноутбук', 'aser'], ['бас', 'гитара', 'invasioн', 'лат', 'bg'], ['смесь', 'грудничок', 'город', 'зеленодольск'], ['shock'], ['санаторий', 'белоруссия', 'лепельский', 'военный'], ['фотохолст']]


In [28]:
test_vocab = [str(test_corpus[i]) for i in range(0, len(test_corpus))]
model.build_vocab(test_corpus, update = True)
model.docvecs.similarity_unseen_docs(model, doc_words1 = train_corpus[0][0], doc_words2 =train_corpus[0][0] )

1.0